In [104]:
# Necessary imports

from pathlib import Path

from torch.utils.data import Dataset  # not the one from PyG!
from torch_geometric.loader import DataLoader
import torch

from torch_geometric.nn import NNConv
from torch.nn.functional import relu, binary_cross_entropy_with_logits, linear
from torch_geometric.utils import add_self_loops, degree
import torch.nn as nn


In [16]:
# MyDataset class for handling file loading

class MyDataset(Dataset):
    def __init__(self, path: Path):
        super().__init__()
        self.graphs = list(path.glob("*.pt"))
    
    def __getitem__(self, idx):
        return torch.load(self.graphs[idx])
    
    def __len__(self) -> int:
        return len(self.graphs)

In [46]:
# Load first batch (batch_1_0)

dataset = MyDataset(Path("./dataset/batch_1_0"))

# Preliminary analysis of dataset behavior

print("Number of graphs in set:", len(dataset))
print("Set of keys in each graph", dataset[0].keys)
print("Number of nodes in first graph:", dataset[0].num_nodes)
print("Number of edges in first graph:", dataset[0].num_edges)
print("Number of node features:", dataset[0].num_node_features)
print("Number of edge features:", dataset[0].num_edge_features)
print("Is this graph undirected? ", dataset[0].is_undirected())

print("Number of edge features:", dataset[0]['y'])

loader = DataLoader(dataset, batch_size=32)

Number of graphs in set: 1000
Set of keys in each graph ['y', 'edge_attr', 'x', 'edge_index']
Number of nodes in first graph: 419
Number of edges in first graph: 4882
Number of node features: 6
Number of edge features: 4
Is this graph undirected?  False
Number of edge features: tensor([1., 1., 1.,  ..., 1., 1., 1.])


In [102]:
class GNN(torch.nn.Module):
    def __init__(self, sample, hidden_layers = [8]):
        super().__init__()

        self.conv = []
        self.linear_weights = torch.randn(2 * sample.num_node_features + sample.num_edge_features)

        if len(hidden_layers) == 0:
            # Case when no hidden layers are passed
            self.conv.append(NNConv(sample.num_node_features, 1))

        else:
            # Case when hidden layers are passed
            self.conv.append(NNConv(sample.num_node_features, hidden_layers[0]))
            for i in range(len(hidden_layers) - 1):
                self.conv.append(NNConv(hidden_layers[i], hidden_layers[i + 1]))    
            self.conv.append(NNConv(hidden_layers[-1], 1))
        
        self.conv_module = torch.nn.ModuleList(self.conv)

    def forward(self, data):

        x, edge_attr, edge_index = data.x, data.edge_attr, data.edge_index
        '''
        # Concatenate the edge attributes with the source node attributes
        src_node_attrs = x[edge_index[0]]
        conv_attributes = torch.cat((edge_attr, src_node_attrs), dim=1)    
        '''
    
        for i in range(len(self.conv)):
            print(i)
            x = self.conv[i](x, edge_index)
            if i < len(self.conv) - 1:
                x = relu(x)

        edge_scores = x

        return edge_scores

In [103]:
device = torch.device('cpu')
model = GNN(dataset[0]).to(device)
data = dataset[0].to(device)
print(list(model.parameters()))
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = binary_cross_entropy_with_logits(out, data.y)
    loss.backward()
    optimizer.step()

[Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True), Parameter containing:
tensor([[ 0.1659, -0.5007,  0.0702, -0.1897,  0.2928,  0.0033],
        [ 0.4252, -0.5678,  0.0354,  0.2101, -0.0033, -0.0868],
        [ 0.4376, -0.5528,  0.4751, -0.1392,  0.1291,  0.4739],
        [-0.1299, -0.4134,  0.0688,  0.2079, -0.6376,  0.1041],
        [-0.0148, -0.1922, -0.2920,  0.5942, -0.1580,  0.0997],
        [ 0.3732,  0.2799, -0.0902,  0.1668,  0.2739,  0.2050],
        [-0.5363, -0.1634,  0.2978,  0.2573,  0.2937, -0.3660],
        [ 0.5720,  0.1038, -0.0672,  0.4645, -0.5442,  0.3502]],
       requires_grad=True), Parameter containing:
tensor([0.], requires_grad=True), Parameter containing:
tensor([[ 0.3674,  0.2895, -0.3118,  0.7534,  0.1908, -0.3774, -0.2028, -0.1939]],
       requires_grad=True)]


TypeError: GNN.forward() missing 2 required positional arguments: 'edge_attr' and 'edge_index'